<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_07/d_HW_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Урок 7. Сверточные нейронные сети для анализа текста
Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [ ]:
my_path = '/content/drive/MyDrive/GB_NLP_04_2022/07/summer_converted.csv'
df = pd.read_csv(my_path)

In [ ]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,"В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в телефоне,приходится пользоваться в ограниченном режиме",2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [ ]:
df.Rating.value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [ ]:
df.Rating = df.Rating - 1

In [ ]:
df.Rating.value_counts()

4    14586
0     2276
3     2138
2      911
1      748
Name: Rating, dtype: int64

In [ ]:
# максимальная длина строки
df['Content'].map(lambda x: len(str(x))).max()

1147

In [ ]:
df['Content'].map(lambda x: len(str(x))).mean()

56.029769107894865

In [ ]:
df['Content'].map(lambda x: len(str(x))).median()

27.0

In [ ]:
df['Content'].map(lambda x: len(str(x))).quantile([.25, .5, .75, 0.9])

0.25000    13.00000
0.50000    27.00000
0.75000    65.00000
0.90000   139.00000
Name: Content, dtype: float64

In [ ]:
max_words = 15000 # словарь
max_len = 140
num_classes = 5

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
!pip install stop_words
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Предобработка

In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [ ]:
train_corpus = " ".join(df["Content"])
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
len(vocabulary)

13017

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test, y_train, y_test = train_test_split(df['Content'], df['Rating'], test_size=0.25)

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test], dtype=np.int32)

In [ ]:
x_train.shape

(15494, 140)

In [ ]:
max_len

140

In [ ]:
df_train

16908                                                                               учиться
8358                                                                     ахуесть приложение
9373                                                                удобнобольшоя создатель
8649                                                                       отлично работать
18174                                     незнаюменить вполне устраивать прилжениеставить 5
                                                ...                                        
9586     писать устройство рута приложение говорить обратный сервис сказать рутанета делать
1372                                                                                       
8666                                                                                  super
9162                                                                                 удобно
5431                    вполне удобный приложение интерфейс интуитивный платёж и

In [ ]:
x_train

array([[   0,    0,    0, ...,    0,    0, 1084],
       [   0,    0,    0, ...,    0, 7654,    1],
       [   0,    0,    0, ...,    0, 3419,  801],
       ...,
       [   0,    0,    0, ...,    0,    0,  441],
       [   0,    0,    0, ...,    0,    0,    2],
       [   0,    0,    0, ...,   38,  152,   95]], dtype=int32)

In [ ]:
#df[df['Rating']==1] = 0
#df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

# Keras model

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Flatten, Dropout, AveragePooling2D, Activation, MaxPooling2D, BatchNormalization, Concatenate
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  

In [ ]:
num_classes = 5
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [ ]:
y_train[0]

array([0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D( 128, 3, kernel_regularizer=tf.keras.regularizers.L1(0.01),
    activity_regularizer=tf.keras.regularizers.L2(0.01) ))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

##добавила регуляризацию в слой conv, так как без нее очень сильно переобучалась модель

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/20
28/28 [==============================] - 0s 18ms/step - loss: 0.9863 - accuracy: 0.7548 - val_loss: 0.8682 - val_accuracy: 0.7465
Epoch 2/20
28/28 [==============================] - 0s 17ms/step - loss: 0.9746 - accuracy: 0.7542 - val_loss: 0.8573 - val_accuracy: 0.7477
Epoch 3/20
28/28 [==============================] - 0s 17ms/step - loss: 0.9612 - accuracy: 0.7604 - val_loss: 0.8445 - val_accuracy: 0.7465
Epoch 4/20
28/28 [==============================] - 0s 17ms/step - loss: 0.9408 - accuracy: 0.7636 - val_loss: 0.8430 - val_accuracy: 0.7516
Epoch 5/20
28/28 [==============================] - 0s 17ms/step - loss: 0.9223 - accuracy: 0.7636 - val_loss: 0.8291 - val_accuracy: 0.7542
Epoch 6/20
28/28 [==============================] - 0s 17ms/step - loss: 0.8938 - accuracy: 0.7684 - val_loss: 0.8226 - val_accuracy: 0.7535
Epoch 7/20
28/28 [==============================] - 0s 17ms/step - loss: 0.8803 - accuracy: 0.7698 - val_loss: 0.8142 - val_accuracy: 0.7497
Epoch 8/20
28

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 0s 5ms/step - loss: 0.7790 - accuracy: 0.7675


Test score: 0.7790473699569702
Test accuracy: 0.7674733996391296


##предобученные векторы

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/184.zip

--2022-06-08 17:30:06--  http://vectors.nlpl.eu/repository/20/184.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 640196018 (611M) [application/zip]
Saving to: ‘184.zip’

184.zip             100%[===================>] 610.54M  12.6MB/s    in 51s     

2022-06-08 17:30:58 (12.1 MB/s) - ‘184.zip’ saved [640196018/640196018]



In [ ]:
!unzip 184.zip -d /184_model

Archive:  184.zip
  inflating: /184_model/meta.json    
  inflating: /184_model/model.bin    
  inflating: /184_model/model.txt    
  inflating: /184_model/README       


In [ ]:
path_to_file = "/184_model/model.txt"

embeddings_index = {}
with open(path_to_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 249319 word vectors.


In [ ]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, weights=[embedding_matrix], output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D( 128, 3, 
                 #kernel_regularizer=tf.keras.regularizers.L1(0.01),
    #activity_regularizer=tf.keras.regularizers.L2(0.01) 
    ))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/20
28/28 [==============================] - 1s 32ms/step - loss: 1.5961 - accuracy: 0.6862 - val_loss: 1.5819 - val_accuracy: 0.6968
Epoch 2/20
28/28 [==============================] - 1s 24ms/step - loss: 1.5683 - accuracy: 0.7079 - val_loss: 1.5551 - val_accuracy: 0.6968
Epoch 3/20
28/28 [==============================] - 1s 24ms/step - loss: 1.5414 - accuracy: 0.7079 - val_loss: 1.5292 - val_accuracy: 0.6968
Epoch 4/20
28/28 [==============================] - 1s 24ms/step - loss: 1.5151 - accuracy: 0.7079 - val_loss: 1.5039 - val_accuracy: 0.6968
Epoch 5/20
28/28 [==============================] - 1s 24ms/step - loss: 1.4896 - accuracy: 0.7079 - val_loss: 1.4794 - val_accuracy: 0.6968
Epoch 6/20
28/28 [==============================] - 1s 24ms/step - loss: 1.4650 - accuracy: 0.7079 - val_loss: 1.4558 - val_accuracy: 0.6968
Epoch 7/20
28/28 [==============================] - 1s 24ms/step - loss: 1.4411 - accuracy: 0.7079 - val_loss: 1.4329 - val_accuracy: 0.6968
Epoch 8/20
28

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 0s 14ms/step - loss: 1.2004 - accuracy: 0.7038


Test score: 1.2004164457321167
Test accuracy: 0.7037754058837891


## в данном случае лучше результат получился на эмбеддингах по умолчанию, но с использованием регуляризации